In [1]:
#!pip install sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
v[0]

0.078222625

In [4]:
import requests 
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
ml_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
assert len(ml_documents) == 375

In [10]:
import numpy as np
from tqdm.auto import tqdm
embeddings = []
for ml_doc in tqdm(ml_documents):
    question = ml_doc['question']
    text = ml_doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))
X = np.array(embeddings)
X.shape

  0%|          | 0/375 [00:00<?, ?it/s]

(375, 768)

In [11]:
np.save('embeddings.npy', X)

In [12]:
scores = X.dot(v)
scores.max()

0.6506573

In [17]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
# retrieved_documents = search_engine.search(v, num_results=5)

In [14]:
import pandas as pd
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'
df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [22]:
def evaluate(ground_truth):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_engine.search(embedding_model.encode(q['question']), num_results=5)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [23]:
evaluate(ground_truth)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}

In [64]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [65]:
for i, doc in tqdm(enumerate(ml_documents)):
    doc['question_text_vector'] = X[i]

0it [00:00, ?it/s]

In [66]:
for doc in tqdm(ml_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [67]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [68]:
results = elastic_search_knn('question_text_vector', v, 'machine-learning-zoomcamp')
results[0]['id']

'ee58a693'

In [69]:
def elastic_search_evaluate(ground_truth):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = elastic_search_knn('question_text_vector', embedding_model.encode(q['question']), 'machine-learning-zoomcamp')
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [70]:
elastic_search_evaluate(ground_truth)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}